In [ ]:
############################################################
# GeoHack 2023 Automated Fire Perimiter
#
#Provides functions to:
# 1.1 Classify Raster
#     > Raster Information
#     > Clip To Specified Bounds
#     > Downsample
#     > Descriminate
#     > Combine
#
#Information:
#     Fire intensity models have utility for various industries.    
#     Cal Fire Intel uses these inputs to generate predictive
#     models that assist in the determination of where the fire
#     will grow, and how fast. Post fire damage assesment teams
#     may use this data to plan rehabilitation of devistated
#     areas based on burn intensity upon exposed flora.
#     
# By: Tony Ramos
# Date: 02/19/2023
#
############################################################

In [61]:
import os
import rasterio
import shapely
import numpy as np
import geopandas

from matplotlib import pyplot




In [62]:
# File Path

work_dir = "./data/"
image = os.path.join(work_dir, "LWIR_QuickMosaic_16-bit_9327.tiff")
poly = os.path.join(work_dir, "HeatPoly.shp")
# work_dir ="Data"
# MillsFire="C:/Users/centr/GeoHack23-WildfirePerimeter/Data/LWIR_QuickMosaic_16-bit_9327.tiff"
OutputFilePath ="./data/"

In [64]:
# Open the raster file using rasterio
with rasterio.open(image) as src:
    # Read the raster data and metadata
    raster_data = src.read(1)  # Read the first band of the raster
    raster_meta = src.meta

    # Classify the raster data
    classified_data = np.where(raster_data < 35000, 0, 1)

# Update the metadata with the new data type and nodata value
raster_meta.update(dtype=rasterio.uint8, nodata=None)

# Write the classified raster to a new file
with rasterio.open(OutputFilePath + 'RasterClass.tif', 'w', **raster_meta) as dst:
    dst.write(classified_data.astype(rasterio.int16), 1)


In [72]:
import rasterio
from rasterio import features
import fiona
import matplotlib.pyplot as plt

# Open the raster file using rasterio
with rasterio.open(image) as src:
    # Read the raster data and metadata
    raster_data = src.read(1)  # Read the first band of the raster
    raster_meta = src.meta

    # Classify the raster data
    classified_data = np.where(raster_data < 35000, 0, 1)

# Update the metadata with the new data type and nodata value
raster_meta.update(dtype=rasterio.int16, nodata=None)

# Write the classified raster to a new file
with rasterio.open(OutputFilePath + 'RasterClass.tif', 'w', **raster_meta) as dst:
    dst.write(classified_data, 1)

# Convert the classified raster to polygons
shapes = features.shapes(classified_data, transform=raster_meta['transform'])

# Write the polygons to a shapefile
with fiona.open(OutputFilePath + 'HeatPoly.shp', 'w', 'ESRI Shapefile', schema={'geometry': 'Polygon', 'properties': {}}) as dst:
    for shape in shapes:
        value = shape[1]
        if value > 0:
            feature = {'geometry': shape[0], 'properties': {}}
            dst.write(feature)

# # (Optional) Display the polygons using matplotlib
# with fiona.open(OutputFilePath + 'HeatPoly.shp', 'r') as src:
#     for i, layer in enumerate(src):
#         if i == 0:
#             x, y = [], []
#             for poly in layer['geometry']['coordinates']:
#                 x += [pt[0] for pt in poly]
#                 y += [pt[1] for pt in poly]
#             plt.plot(x, y, 'k.')
#             plt.axis('equal')
#             plt.show()


In [75]:
import geopandas as shp
from shapely.ops import unary_union
from shapely.geometry import MultiPolygon

# Load your polygons into a GeoDataFrame
polygons = shp.read_file(work_dir + 'HeatPoly.shp')

# Set the distance within which you want to aggregate polygons
distance = 1  # in meters

# Create a buffer around each polygon using the distance
buffered = polygons.geometry.buffer(distance)

# Group buffered polygons that intersect with each other
groups = buffered.unary_union

# Convert the grouped polygons back to a GeoDataFrame
if isinstance(groups, MultiPolygon):
    grouped_polygons = gpd.GeoDataFrame(
        {'geometry': [polygon for polygon in groups]},
        crs=polygons.crs
    )
else:
    grouped_polygons = gpd.GeoDataFrame(
        {'geometry': [groups]},
        crs=polygons.crs
    )

# Save the aggregated polygons to a shapefile
grouped_polygons.to_file(OutputFilePath + 'Aggrigate.shp')
